# Sample voilá dashboard

Plotting functions with altair,
inputs select from available patient data.

Can choose between heart rate and blood pressure charts.

In [ ]:
from jupyter_health import JupyterHealthCHClient

ch_client = JupyterHealthCHClient()

# df = ch_client.fetch_data_frame("test-user")

In [ ]:
import altair as alt

# common formatting for date axes
_date_x = alt.X(
    "effective_time_frame_date_time",
    title="date",
    axis=alt.Axis(
        format="%Y-%m",
        labelAngle=30,
        tickCount={"interval": "month", "step": 3},
    ),
)


def plot_patient_blood_pressure(patient_df):
    """plot blood pressure, given a patient data frame, as returned by get_patient_data"""
    bp = patient_df[patient_df.resource_type == "BLOOD_PRESSURE"]
    # get set1 color scale
    # https://vega.github.io/vega/docs/schemes/#set1
    set1_colors = [
        "#e41a1c",
        "#377eb8",
        "#4daf4a",
        "#984ea3",
        "#ff7f00",
        "#ffff33",
        "#a65628",
        "#f781bf",
        "#999999",
    ]
    scale = alt.Scale(domain=["systolic", "diastolic"], range=set1_colors[:2])
    charts = [
        alt.Chart(bp, title="blood pressure")
        .transform_calculate(systolic="'systolic'", diastolic="'diastolic'")
        .mark_line(point=True)
        .encode(
            x=_date_x,
            y=alt.Y(f"{which}_blood_pressure_value", title="mmHg"),
            color=alt.Color(f"{which}:N", scale=scale, title=""),
            tooltip=[
                alt.Tooltip("effective_time_frame_date_time", title="date"),
                alt.Tooltip("systolic_blood_pressure_value", title="Systolic"),
                alt.Tooltip("diastolic_blood_pressure_value", title="Diastolic"),
            ],
        )
        for which in ("systolic", "diastolic")
    ]
    return alt.layer(*charts).interactive().configure_point(size=50)


# plot_patient_blood_pressure(df)

In [ ]:
def plot_patient_heart_rate(patient_df):
    """plot heart rate, given a patient data frame, as returned by get_patient_data"""
    heart_rate = patient_df[patient_df.resource_type == "HEART_RATE"]
    return alt.Chart(heart_rate, title="Heart Rate").mark_line(point=True).encode(
        x=_date_x,
        y=alt.Y("heart_rate_value", title="bpm"),
        tooltip=[
            alt.Tooltip("effective_time_frame_date_time", title="date"),
            alt.Tooltip("heart_rate_value", title="heart rate"),
            alt.Tooltip("heart_rate_unit", title="unit"),
        ],
    ).interactive().configure_point(size=50)

# plot_patient_heart_rate(df)

In [ ]:
all_patients = ch_client.list_patients()

In [ ]:
import time
from datetime import date
from functools import lru_cache

import ipywidgets as W
from commonhealth_cloud_storage_client.errors import UserNotConsented
from IPython.display import Markdown, display


@lru_cache
def _cache_fetch_data(patient_id):
    """Cache fetching the same data"""
    return ch_client.fetch_data_frame(patient_id)

# create the interactive chart


@W.interact(
    # pick patient by id
    patient_id=all_patients,
    # start date is a bit silly, but gives more control
    start_date=W.DatePicker(value=date(2022, 1, 1)),
    # chart picker
    chart={
        "blood pressure": plot_patient_blood_pressure,
        "heart rate": plot_patient_heart_rate,
    },
)
def plot_patient(
    patient_id=all_patients[0],
    start_date=date(2022, 1, 1),
    chart=plot_patient_blood_pressure,
):
    try:
        df = _cache_fetch_data(patient_id)
    except UserNotConsented:
        display(Markdown(f"**{patient_id} has not consented to sharing data**"))
        return
    if start_date:
        df = df[df.effective_time_frame_date_time.dt.date >= start_date]
    display(chart(df))
    # utterly bizarre: voilá hangs if this returns too quickly
    time.sleep(0.5)